In [2]:
%autosave 0
%matplotlib inline

Autosave disabled


In [3]:
import codecs
import pymarc
import sqlite3

In [4]:
INPUT_MARCXML_FILE = "/src/input/metadata/Perth images dataset (MARC21).xml"

In [16]:
class Loader:
    
    def __init__(self, uri):
        self.db = sqlite3.connect(uri)

    def load_xml(self, xml_file):
        handler = DatabaseHandler()
        pymarc.parse_xml(xml_file, handler)
        self.db.commit()

    def load_marc(self, marc_file):
        handler = DatabaseHandler()
        for record in pymarc.MARCReader(file(marc_file)):
            handler.process_record(record)
        self.db.commit()

In [18]:
class DatabaseHandler(pymarc.XmlHandler):
    
    def __init__(self):
        self._record_count = 0
        
    def process_record(self, marc_record):
        self._record_count += 1
        print(self._record_count)

    r = Record()
    r.leader = marc_record.leader
    field_position = -1
    for marc_field in marc_record:
        field_position += 1
    if marc_field.is_control_field():
        f = ControlField()
        f.tag = marc_field.tag
        f.value = marc_field.data
        f.position = field_position
        r.control_fields.append(f)
    else:
        f = DataField()
        f.tag = marc_field.tag
        f.indicator1 = marc_field.indicators[0]
        f.indicator2 = marc_field.indicators[1]
        f.position = field_position
        subfield_position = -1
    for marc_subfield in marc_field:
        subfield_position += 1
        sf = Subfield()
        sf.code=marc_subfield[0]
        sf.value=marc_subfield[1]
        sf.position = subfield_position
        f.subfields.append(sf)
    r.data_fields.append(f)

NameError: name 'Record' is not defined